# TP04 — Differential Kinematics (Session 1: Exercises 2-4)

**GEII3 — Industrie 4.0: Robotique | Lab Session 4 - Part 1**

Continuation: Geometric Jacobian, numerical methods, and singularity analysis.

---

In [ ]:
# Setup (copy from main or re-run)
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, sqrt
from scipy.linalg import svd, det, pinv

try:
    import roboticstoolbox as rtb
    RTB_AVAILABLE = True
except ImportError:
    RTB_AVAILABLE = False

np.set_printoptions(precision=4, suppress=True)
print("✓ Session 1 ready")

---

# Exercise 2: Geometric Jacobian Approach

**Estimated time**: 50 minutes

## Objective

Compute Jacobian using **geometric method** (screw theory) and compare with analytical.

## Task 2.1: Theory Review (15 min)

### Geometric Jacobian Formula

For joint $i$, the $i$-th column of Jacobian is:

**Revolute joint**:
$$J_i = \begin{bmatrix}
\mathbf{z}_{i-1} \times (\mathbf{p}_n - \mathbf{p}_{i-1}) \\
\mathbf{z}_{i-1}
\end{bmatrix}$$

**Prismatic joint**:
$$J_i = \begin{bmatrix}
\mathbf{z}_{i-1} \\
\mathbf{0}
\end{bmatrix}$$

where:
- $\mathbf{z}_{i-1}$: axis of motion for joint $i$ (in world frame)
- $\mathbf{p}_n$: end-effector position
- $\mathbf{p}_{i-1}$: position of joint $i$

### Example: 2R Planar

**Joint 1** (at origin):
- $\mathbf{z}_0 = [0, 0, 1]^T$ (rotation about Z)
- $\mathbf{p}_0 = [0, 0, 0]^T$
- $\mathbf{p}_n = [x, y, 0]^T$

$$J_1 = \begin{bmatrix}
[0,0,1] \times [x, y, 0] \\
[0, 0, 1]
\end{bmatrix} = \begin{bmatrix}
-y \\
x \\
0 \\
0 \\
0 \\
1
\end{bmatrix}$$

For 2D planar, we only need first 2 rows (linear velocity in XY).

---

## Task 2.2: Implementation for 2R (20 min)

In [ ]:
L1, L2 = 0.4, 0.3
dh_2r = [(0,0,L1,0,'r'), (0,0,L2,0,'r')]

def jacobian_2r_geometric(q, L1, L2):
    """
    Geometric Jacobian for 2R planar.
    Returns 2×2 (only XY velocities for planar robot).
    """
    q1, q2 = q[0], q[1]
    
    # End-effector position
    x_n = L1*cos(q1) + L2*cos(q1+q2)
    y_n = L1*sin(q1) + L2*sin(q1+q2)
    
    # Joint 1 at origin
    z0 = np.array([0, 0, 1])
    p0 = np.array([0, 0, 0])
    pn = np.array([x_n, y_n, 0])
    
    # Column 1: z0 × (pn - p0)
    v1 = np.cross(z0, pn - p0)
    
    # Joint 2 position
    p1 = np.array([L1*cos(q1), L1*sin(q1), 0])
    z1 = np.array([0, 0, 1])  # Still Z axis
    
    # Column 2: z1 × (pn - p1)
    v2 = np.cross(z1, pn - p1)
    
    # Extract XY components only (planar)
    J = np.array([
        [v1[0], v2[0]],
        [v1[1], v2[1]]
    ])
    
    return J


# Compare analytical vs geometric
print("\n" + "="*70)
print("  ANALYTICAL vs GEOMETRIC JACOBIAN")
print("="*70)

q_test = np.array([np.deg2rad(30), np.deg2rad(45)])

J_ana = jacobian_2r_analytical(q_test[0], q_test[1], L1, L2)
J_geo = jacobian_2r_geometric(q_test, L1, L2)

print(f"\nConfiguration: q = {np.rad2deg(q_test).astype(int)}°")

print("\nAnalytical Jacobian:")
print(J_ana)

print("\nGeometric Jacobian:")
print(J_geo)

print("\nDifference:")
diff = np.abs(J_ana - J_geo)
print(diff)
print(f"\nMax difference: {np.max(diff):.2e}")

if np.max(diff) < 1e-10:
    print("✓ Perfect match!")
else:
    print("⚠ Methods differ - check implementation")

## Task 2.3: Geometric Jacobian for 3R (15 min)

Extend to 3R planar robot with orientation.

In [ ]:
L1_3r, L2_3r, L3_3r = 0.3, 0.3, 0.2
dh_3r = [(0,0,L1_3r,0,'r'), (0,0,L2_3r,0,'r'), (0,0,L3_3r,0,'r')]

# Use generic geometric Jacobian function
q_3r = np.array([np.deg2rad(30), np.deg2rad(45), np.deg2rad(-30)])

print("\n" + "="*70)
print("  3R PLANAR - GEOMETRIC JACOBIAN")
print("="*70)

J_geo_3r = jacobian_geometric(dh_3r, q_3r)

print(f"\nConfiguration: q = {np.rad2deg(q_3r).astype(int)}°")
print(f"\nFull Jacobian (6×3):")
print(J_geo_3r)

print(f"\nLinear velocity part (first 3 rows):")
print(J_geo_3r[:3, :])

print(f"\nAngular velocity part (last 3 rows):")
print(J_geo_3r[3:, :])

# For planar robot, all joints rotate about Z
print(f"\nNote: All angular velocity rows should be [0,0,1] or [0,0,0]")

---

# Exercise 3: Numerical Jacobian & Validation

**Estimated time**: 40 minutes

## Task 3.1: Numerical Implementation (20 min)

In [ ]:
print("\n" + "="*70)
print("  NUMERICAL JACOBIAN - FINITE DIFFERENCES")
print("="*70)

# Test with 3R
q_test = np.array([np.deg2rad(30), np.deg2rad(45), np.deg2rad(-30)])

# Different epsilon values
epsilons = [1e-4, 1e-6, 1e-8, 1e-10]

# Reference: geometric Jacobian
J_ref = jacobian_geometric(dh_3r, q_test)

print(f"\nConfiguration: q = {np.rad2deg(q_test).astype(int)}°")
print(f"\nReference (Geometric) Jacobian:")
print(J_ref)

print(f"\n{'Epsilon':<12} {'Max Error':<15} {'Condition'}")
print("-" * 45)

for eps in epsilons:
    J_num = jacobian_numerical(dh_3r, q_test, epsilon=eps)
    error = np.max(np.abs(J_num - J_ref))
    
    if error < 1e-3:
        condition = "✓ Good"
    elif error < 1e-2:
        condition = "~ Acceptable"
    else:
        condition = "✗ Poor"
    
    print(f"{eps:<12.0e} {error:<15.2e} {condition}")

print("\n** Observation: eps=1e-6 typically gives best balance")

## Task 3.2: RTB Validation (20 min)

In [ ]:
if RTB_AVAILABLE:
    from roboticstoolbox import DHRobot, RevoluteDH
    
    # Create 3R robot
    robot_3r = DHRobot([
        RevoluteDH(a=0,      d=0, alpha=0),
        RevoluteDH(a=L1_3r,  d=0, alpha=0),
        RevoluteDH(a=L2_3r,  d=0, alpha=0)
    ], name="3R Planar")
    
    print("\n" + "="*70)
    print("  VALIDATION WITH ROBOTICS TOOLBOX")
    print("="*70)
    
    # Compute Jacobian with RTB
    J_rtb = robot_3r.jacob0(q_test)  # Jacobian in base frame
    
    print(f"\nRTB Jacobian (6×3):")
    print(J_rtb)
    
    # Compare with our methods
    J_geo = jacobian_geometric(dh_3r, q_test)
    J_num = jacobian_numerical(dh_3r, q_test)
    
    print("\n--- Comparison with RTB ---")
    
    error_geo = np.max(np.abs(J_geo - J_rtb))
    error_num = np.max(np.abs(J_num - J_rtb))
    
    print(f"Geometric vs RTB: max error = {error_geo:.2e}")
    print(f"Numerical vs RTB: max error = {error_num:.2e}")
    
    if max(error_geo, error_num) < 1e-4:
        print("\n✓ All methods agree!")
    else:
        print("\n⚠ Check frame conventions (base vs tool)")
    
    # Manipulability
    m_rtb = robot_3r.manipulability(q_test)
    m_ours = manipulability(J_geo)
    
    print(f"\nManipulability (RTB): {m_rtb:.6f}")
    print(f"Manipulability (ours): {m_ours:.6f}")
    
else:
    print("RTB not available - skipping validation")

---

# Exercise 4: Singularity Analysis

**Estimated time**: 70 minutes

## Task 4.1: Theory - Types of Singularities (15 min)

### Three Types of Singularities

**1. Workspace Boundary Singularities**
- Robot at maximum or minimum reach
- Example: 2R fully extended or folded

**2. Internal Singularities**
- Occur inside workspace
- Two or more joint axes align
- Example: 3R wrist aligned with shoulder

**3. Wrist Singularities** (6-DOF robots)
- Last 3 joint axes align
- Example: PUMA with q5 = 0

### Detection Methods

1. **Determinant**: $\det(J) = 0$ (square Jacobian)
2. **Rank**: $\text{rank}(J) < \min(m, n)$
3. **Singular values**: $\sigma_{\min} \approx 0$
4. **Manipulability**: $w = \sqrt{\det(J J^T)} \approx 0$

---

## Task 4.2: Singularity Map for 2R (25 min)

Visualize manipulability across configuration space.

In [ ]:
print("\n" + "="*70)
print("  2R SINGULARITY ANALYSIS")
print("="*70)

# Sample configuration space
n_samples = 50
q1_range = np.linspace(-pi, pi, n_samples)
q2_range = np.linspace(-pi, pi, n_samples)

Q1, Q2 = np.meshgrid(q1_range, q2_range)
M = np.zeros_like(Q1)  # Manipulability
D = np.zeros_like(Q1)  # Determinant

print("\nComputing manipulability map...")
for i in range(n_samples):
    for j in range(n_samples):
        q = np.array([Q1[i,j], Q2[i,j]])
        J = jacobian_2r_geometric(q, L1, L2)
        M[i,j] = abs(det(J))  # Manipulability for 2D
        D[i,j] = det(J)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Manipulability map
im1 = axes[0].contourf(np.rad2deg(Q1), np.rad2deg(Q2), M, levels=20, cmap='viridis')
axes[0].contour(np.rad2deg(Q1), np.rad2deg(Q2), M, levels=[0.01], colors='red', linewidths=2)
axes[0].set_xlabel('q1 (deg)', fontsize=12)
axes[0].set_ylabel('q2 (deg)', fontsize=12)
axes[0].set_title('Manipulability Map\n(red line: singularity threshold)', 
                   fontsize=13, fontweight='bold')
plt.colorbar(im1, ax=axes[0], label='|det(J)|')
axes[0].grid(True, alpha=0.3)

# Determinant sign
im2 = axes[1].contourf(np.rad2deg(Q1), np.rad2deg(Q2), np.sign(D), 
                        levels=[-1, 0, 1], cmap='RdBu', alpha=0.6)
axes[1].contour(np.rad2deg(Q1), np.rad2deg(Q2), D, levels=[0], 
                colors='black', linewidths=3, linestyles='--')
axes[1].set_xlabel('q1 (deg)', fontsize=12)
axes[1].set_ylabel('q2 (deg)', fontsize=12)
axes[1].set_title('Determinant Sign\n(black dashed: singularities)', 
                   fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Identify singular configurations
singular_mask = M < 0.01
n_singular = np.sum(singular_mask)
percentage = 100 * n_singular / (n_samples**2)

print(f"\nSingular configurations: {n_singular}/{n_samples**2} ({percentage:.1f}%)")
print(f"\nTypical singularities for 2R:")
print(f"  - q2 = 0° (fully extended)")
print(f"  - q2 = ±180° (fully folded)")

## Task 4.3: Specific Singular Configurations (30 min)

Analyze behavior at and near singularities.

In [ ]:
print("\n" + "="*70)
print("  BEHAVIOR AT SINGULARITIES")
print("="*70)

singular_configs = [
    (np.array([0, 0]), "Fully extended"),
    (np.array([0, np.pi]), "Fully folded"),
    (np.array([np.pi/4, 0]), "Extended at 45°"),
    (np.array([np.pi/2, np.pi]), "Folded at 90°"),
]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, (q, desc) in enumerate(singular_configs):
    # Compute Jacobian
    J = jacobian_2r_geometric(q, L1, L2)
    
    # Forward kinematics
    _, T_list = fk_dh(dh_2r, q)
    
    # Analysis
    det_J = det(J)
    U, s, Vt = svd(J)
    
    print(f"\n[{desc}] q = {np.rad2deg(q).astype(int)}°")
    print(f"  Jacobian:\n{J}")
    print(f"  det(J) = {det_J:.6f}")
    print(f"  Singular values: {s}")
    print(f"  Rank: {np.linalg.matrix_rank(J)}")
    
    if abs(det_J) < 1e-6:
        print(f"  ⚠ SINGULAR! Lost direction:")
        # Null space direction
        null_dir = Vt[-1, :]  # Last row of V^T
        print(f"    Null space: {null_dir}")
    
    # Visualize
    plot_robot_2d(T_list, axes[idx], 
                  title=f"{desc}\ndet(J)={det_J:.4f}",
                  J=J)

plt.tight_layout()
plt.show()

print("\n✓ Exercise 4 complete - singularities identified!")
print("\n--- End of Session 1 ---")